# load packages

In [164]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
import time

from summarytools import dfSummary

import os
from tqdm import tqdm


# 데이터 로드

In [172]:
train = pd.read_csv("./datasets/train_Preprocessed_noegg.csv", index_col=0)
test = pd.read_csv("./datasets/test_Preprocessed_noegg.csv", index_col=0)

In [173]:
train_subset = pd.read_csv("./datasets/train_Preprocessed_noegg_subset.csv", index_col=0)
test_subset = pd.read_csv("./datasets/test_Preprocessed_noegg_subset.csv", index_col=0)

In [174]:
# 수정된 컬럼명으로 X, y 분리
X = train.drop(columns=['임신 성공 여부'])
y = train['임신 성공 여부']

# Gridsearch
- best params : {'depth': 6, 'iterations': 500, 'learning_rate': 0.05}

In [175]:
# 3️⃣ 모델별 하이퍼파라미터 후보군 설정
param_grid = {
    # "CatBoost": {
    #     "depth": [6],
    #     "learning_rate": [0.05],
    #     "iterations": [500]
    # },
    # "XGBoost": {
    #     "max_depth": [1, 2, 3, 4, 5, 6, 7, 8],
    #     "learning_rate": [0.075, 0.08, 0.085],
    #     "n_estimators": [300, 400, 500, 600, 700, 800]
    # },
    "XGBoost" : {'learning_rate': [0.08], 'max_depth': [3], 'n_estimators': [300]}

    # "RandomForest": {
    #     "max_depth": [10, 20, 30, 40, 50],
    #     "n_estimators": [100, 300, 500, 700, 1000],
    #     "min_samples_split": [2, 5, 10, 15, 20]
    # }
}

# 4️⃣ 모델 정의
models = {
    # "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42),
    # "RandomForest": RandomForestClassifier(random_state=42)
}

In [176]:
# 🔹 병렬 처리 설정
N_JOBS = 7  # CPU 코어 개수 (변경 가능)

# 5️⃣ 10-Fold Cross Validation 및 GridSearch
best_models = {}
cv_results = {}
auc_results = {}  # ✅ AUC 점수 저장할 딕셔너리 추가

start_time = time.time()  # 전체 실행 시간 측정 시작

for model_name, model in models.items():
    print(f"\n🚀 {model_name} 모델 GridSearch 시작...")
    model_start = time.time()  # 개별 모델 실행 시간 측정

    grid_search = GridSearchCV(
        model, param_grid[model_name], 
        scoring="roc_auc", cv=3, n_jobs=N_JOBS, verbose=3
    )
    
    if model_name == "CatBoost":
        # ✅ tqdm으로 GridSearch 진행 상황 표시
        with tqdm(total=len(param_grid[model_name]["depth"]) * 
                        len(param_grid[model_name]["learning_rate"]) * 
                        len(param_grid[model_name]["iterations"]), desc=f"🔍 GridSearch {model_name}") as pbar:
            grid_search.fit(X, y)
            pbar.update()
    elif model_name == "XGBoost":
        # ✅ tqdm으로 GridSearch 진행 상황 표시
        with tqdm(total=len(param_grid[model_name]["max_depth"]) * 
                        len(param_grid[model_name]["learning_rate"]) * 
                        len(param_grid[model_name]["n_estimators"]), desc=f"🔍 GridSearch {model_name}") as pbar:
            grid_search.fit(X, y)
            pbar.update()
    elif model_name == "RandomForest":
        with tqdm(total=len(param_grid[model_name]["max_depth"]) * 
                        len(param_grid[model_name]["n_estimators"]) * 
                        len(param_grid[model_name]["min_samples_split"]), desc=f"🔍 GridSearch {model_name}") as pbar:
            grid_search.fit(X, y)
            pbar.update()

    # 최적 모델 저장
    best_models[model_name] = grid_search.best_estimator_
    print(f"✅ {model_name} 최적 파라미터: {grid_search.best_params_}")

    # ✅ 10-Fold Cross Validation 진행 상황을 tqdm으로 표시
    print(f"📌 {model_name} 10-Fold Cross Validation 시작...")
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    auc_scores = []

    for fold_idx, (train_idx, val_idx) in tqdm(enumerate(skf.split(X, y), 1), total=4, desc=f"🔄 {model_name} CV Progress"):
        X_t, X_v = X.iloc[train_idx], X.iloc[val_idx]
        y_t, y_v = y.iloc[train_idx], y.iloc[val_idx]

        model = grid_search.best_estimator_
        model.fit(X_t, y_t)
        y_pred = model.predict_proba(X_v)[:, 1]
        auc = roc_auc_score(y_v, y_pred)
        auc_scores.append(auc)

    # ✅ AUC 점수 정렬 후 출력
    sorted_auc = sorted(auc_scores)  # 정렬하면 점수의 변동성을 한눈에 파악 가능
    print(f"\n📊 {model_name} 4-Fold AUC Scores (정렬됨) = {sorted_auc}")

    # ✅ AUC 평균 및 표준편차 저장
    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)    
    cv_results[model_name] = (mean_auc, std_auc)
    auc_results[model_name] = auc_scores  # ✅ 개별 AUC 점수 저장

    print(f"📊 {model_name} CV AUC: 평균={mean_auc:.6f}, 표준편차={std_auc:.6f}")

    model_end = time.time()
    print(f"🕒 {model_name} 학습 완료! 소요 시간: {model_end - model_start:.2f}초\n")

total_time = time.time() - start_time
print(f"\n🚀 전체 학습 완료! 총 소요 시간: {total_time:.2f}초")


🚀 XGBoost 모델 GridSearch 시작...


🔍 GridSearch XGBoost:   0%|          | 0/1 [00:00<?, ?it/s]

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END learning_rate=0.08, max_depth=3, n_estimators=300;, score=0.803 total time=   4.1s
[CV 3/3] END learning_rate=0.08, max_depth=3, n_estimators=300;, score=0.804 total time=   4.1s
[CV 2/3] END learning_rate=0.08, max_depth=3, n_estimators=300;, score=0.803 total time=   4.2s


🔍 GridSearch XGBoost: 100%|██████████| 1/1 [00:07<00:00,  7.85s/it]


✅ XGBoost 최적 파라미터: {'learning_rate': 0.08, 'max_depth': 3, 'n_estimators': 300}
📌 XGBoost 10-Fold Cross Validation 시작...


🔄 XGBoost CV Progress: 100%|██████████| 4/4 [00:06<00:00,  1.73s/it]


📊 XGBoost 4-Fold AUC Scores (정렬됨) = [0.800815544000967, 0.8035024636255388, 0.8039779432179481, 0.8056574439507604]
📊 XGBoost CV AUC: 평균=0.803488, 표준편차=0.001738
🕒 XGBoost 학습 완료! 소요 시간: 14.76초


🚀 전체 학습 완료! 총 소요 시간: 14.77초


In [ ]:
# 수정된 컬럼명으로 X, y 분리
X = train_subset.drop(columns=['임신 성공 여부'])
y = train_subset['임신 성공 여부']

In [178]:
# 🔹 병렬 처리 설정
N_JOBS = 7  # CPU 코어 개수 (변경 가능)

# 5️⃣ 10-Fold Cross Validation 및 GridSearch
best_models = {}
cv_results = {}
auc_results = {}  # ✅ AUC 점수 저장할 딕셔너리 추가

start_time = time.time()  # 전체 실행 시간 측정 시작

for model_name, model in models.items():
    print(f"\n🚀 {model_name} 모델 GridSearch 시작...")
    model_start = time.time()  # 개별 모델 실행 시간 측정

    grid_search = GridSearchCV(
        model, param_grid[model_name], 
        scoring="roc_auc", cv=3, n_jobs=N_JOBS, verbose=3
    )
    
    if model_name == "CatBoost":
        # ✅ tqdm으로 GridSearch 진행 상황 표시
        with tqdm(total=len(param_grid[model_name]["depth"]) * 
                        len(param_grid[model_name]["learning_rate"]) * 
                        len(param_grid[model_name]["iterations"]), desc=f"🔍 GridSearch {model_name}") as pbar:
            grid_search.fit(X, y)
            pbar.update()
    elif model_name == "XGBoost":
        # ✅ tqdm으로 GridSearch 진행 상황 표시
        with tqdm(total=len(param_grid[model_name]["max_depth"]) * 
                        len(param_grid[model_name]["learning_rate"]) * 
                        len(param_grid[model_name]["n_estimators"]), desc=f"🔍 GridSearch {model_name}") as pbar:
            grid_search.fit(X, y)
            pbar.update()
    elif model_name == "RandomForest":
        with tqdm(total=len(param_grid[model_name]["max_depth"]) * 
                        len(param_grid[model_name]["n_estimators"]) * 
                        len(param_grid[model_name]["min_samples_split"]), desc=f"🔍 GridSearch {model_name}") as pbar:
            grid_search.fit(X, y)
            pbar.update()

    # 최적 모델 저장
    best_models[model_name] = grid_search.best_estimator_
    print(f"✅ {model_name} 최적 파라미터: {grid_search.best_params_}")

    # ✅ 10-Fold Cross Validation 진행 상황을 tqdm으로 표시
    print(f"📌 {model_name} 10-Fold Cross Validation 시작...")
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    auc_scores = []

    for fold_idx, (train_idx, val_idx) in tqdm(enumerate(skf.split(X, y), 1), total=4, desc=f"🔄 {model_name} CV Progress"):
        X_t, X_v = X.iloc[train_idx], X.iloc[val_idx]
        y_t, y_v = y.iloc[train_idx], y.iloc[val_idx]

        model = grid_search.best_estimator_
        model.fit(X_t, y_t)
        y_pred = model.predict_proba(X_v)[:, 1]
        auc = roc_auc_score(y_v, y_pred)
        auc_scores.append(auc)

    # ✅ AUC 점수 정렬 후 출력
    sorted_auc = sorted(auc_scores)  # 정렬하면 점수의 변동성을 한눈에 파악 가능
    print(f"\n📊 {model_name} 4-Fold AUC Scores (정렬됨) = {sorted_auc}")

    # ✅ AUC 평균 및 표준편차 저장
    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)    
    cv_results[model_name] = (mean_auc, std_auc)
    auc_results[model_name] = auc_scores  # ✅ 개별 AUC 점수 저장

    print(f"📊 {model_name} CV AUC: 평균={mean_auc:.6f}, 표준편차={std_auc:.6f}")

    model_end = time.time()
    print(f"🕒 {model_name} 학습 완료! 소요 시간: {model_end - model_start:.2f}초\n")

total_time = time.time() - start_time
print(f"\n🚀 전체 학습 완료! 총 소요 시간: {total_time:.2f}초")


🚀 XGBoost 모델 GridSearch 시작...


🔍 GridSearch XGBoost:   0%|          | 0/1 [00:00<?, ?it/s]

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 2/3] END learning_rate=0.08, max_depth=3, n_estimators=300;, score=0.710 total time=   1.0s
[CV 3/3] END learning_rate=0.08, max_depth=3, n_estimators=300;, score=0.706 total time=   1.1s
[CV 1/3] END learning_rate=0.08, max_depth=3, n_estimators=300;, score=0.705 total time=   1.1s


🔍 GridSearch XGBoost: 100%|██████████| 1/1 [00:02<00:00,  2.76s/it]


✅ XGBoost 최적 파라미터: {'learning_rate': 0.08, 'max_depth': 3, 'n_estimators': 300}
📌 XGBoost 10-Fold Cross Validation 시작...


🔄 XGBoost CV Progress: 100%|██████████| 4/4 [00:02<00:00,  1.61it/s]


📊 XGBoost 4-Fold AUC Scores (정렬됨) = [0.7022798626307379, 0.7043853665045158, 0.7094010200311487, 0.7105104445101108]
📊 XGBoost CV AUC: 평균=0.706644, 표준편차=0.003417
🕒 XGBoost 학습 완료! 소요 시간: 5.25초


🚀 전체 학습 완료! 총 소요 시간: 5.26초


In [179]:
# ✅ AUC 결과를 DataFrame으로 변환하여 CSV로 저장 (원하면 확인 가능)
auc_df = pd.DataFrame(auc_results)
display(auc_df)

,XGBoost
0,0.709401
1,0.702280
2,0.710510
3,0.704385


# Finalize

In [159]:
# 7️⃣ 최적 하이퍼파라미터로 최종 모델 학습 (전체 데이터 사용)
print("\n🚀 최적 하이퍼파라미터로 Final Model 학습 시작...")

# 전체 데이터로 다시 X, y 설정
X = train.drop(columns=['임신 성공 여부'])  # ✅ 컬럼명에 공백 없애기
y = train['임신 성공 여부']

# 최적 파라미터 가져오기
# best_params = best_models["CatBoost"].get_params()
best_params = {'depth': 6, 'iterations': 500, 'learning_rate': 0.05}
best_params.update({"verbose": 0, "random_state": 42, "thread_count": N_JOBS})  # ✅ 불필요한 verbose 제거

# 최종 모델 훈련
final_model = CatBoostClassifier(**best_params)
final_model.fit(X, y)

print("✅ Final Model 학습 완료!")

# 8️⃣ Test 데이터에 대해 predict_proba 수행
print("\n🚀 Test 데이터 예측 (predict_proba) 시작...")

# test 데이터 로드 (파일명 확인 필요)
# test = pd.read_csv("./test_Preprocessed_paper_col.csv", index_col=0)
X_test = test.copy()  # 예측을 위한 X_test 준비

# 예측 수행 (확률 값)
test_preds = final_model.predict_proba(X_test)[:, 1]  # 양성(1)의 확률값 가져오기



🚀 최적 하이퍼파라미터로 Final Model 학습 시작...
✅ Final Model 학습 완료!

🚀 Test 데이터 예측 (predict_proba) 시작...


In [59]:
# 7️⃣ 최적 하이퍼파라미터로 최종 모델 학습 (전체 데이터 사용)
print("\n🚀 최적 하이퍼파라미터로 Final Model 학습 시작...")

# 전체 데이터로 다시 X, y 설정
X = train.drop(columns=['임신 성공 여부'])  # ✅ 컬럼명에 공백 없애기
y = train['임신 성공 여부']

# 최적 파라미터 가져오기
best_params = best_models["XGBoost"].get_params()
best_params.update({"verbose": 0, "random_state": 42, "thread_count": N_JOBS})  # ✅ 불필요한 verbose 제거

# # 최종 모델 훈련
final_model = XGBClassifier(**best_params)
final_model.fit(X, y)

print("✅ Final Model 학습 완료!")

# 8️⃣ Test 데이터에 대해 predict_proba 수행
print("\n🚀 Test 데이터 예측 (predict_proba) 시작...")

# test 데이터 로드 (파일명 확인 필요)
# test = pd.read_csv("./test_Preprocessed_paper_col.csv", index_col=0)
X_test = test.copy()  # 예측을 위한 X_test 준비

# 예측 수행 (확률 값)
test_preds = final_model.predict_proba(X_test)[:, 1]  # 양성(1)의 확률값 가져오기



🚀 최적 하이퍼파라미터로 Final Model 학습 시작...


/Users/dowenty/anaconda3/envs/aimers/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:22:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "thread_count", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Final Model 학습 완료!

🚀 Test 데이터 예측 (predict_proba) 시작...


## Finalize for subset

In [160]:
# 7️⃣ 최적 하이퍼파라미터로 최종 모델 학습 (전체 데이터 사용)
print("\n🚀 최적 하이퍼파라미터로 Final Model 학습 시작...")

# 전체 데이터로 다시 X, y 설정
X = train_subset.drop(columns=['임신 성공 여부'])  # ✅ 컬럼명에 공백 없애기
y = train_subset['임신 성공 여부']

# 최적 파라미터 가져오기
best_params = {'depth': 6, 'iterations': 500, 'learning_rate': 0.05}
best_params.update({"verbose": 0, "random_state": 42, "thread_count": N_JOBS})  # ✅ 불필요한 verbose 제거

# 최종 모델 훈련
final_model = CatBoostClassifier(**best_params)
final_model.fit(X, y)

print("✅ Final Model 학습 완료!")

# 8️⃣ Test 데이터에 대해 predict_proba 수행
print("\n🚀 Test 데이터 예측 (predict_proba) 시작...")

# test 데이터 로드 (파일명 확인 필요)
# test = pd.read_csv("./test_Preprocessed_paper_col.csv", index_col=0)
X_test = test_subset.copy()  # 예측을 위한 X_test 준비

# 예측 수행 (확률 값)
test_preds_subset = final_model.predict_proba(X_test)[:, 1]  # 양성(1)의 확률값 가져오기



🚀 최적 하이퍼파라미터로 Final Model 학습 시작...
✅ Final Model 학습 완료!

🚀 Test 데이터 예측 (predict_proba) 시작...


In [161]:
test_preds_df = pd.DataFrame(test_preds)
test_preds_df = test_preds_df.set_index(test.reset_index()['index'])

test_preds_subset_df = pd.DataFrame(test_preds_subset)
test_preds_subset_df = test_preds_subset_df.set_index(test_subset.reset_index()['index'])

test_preds = pd.concat([test_preds_df, test_preds_subset_df]).sort_index()

# Submission

In [162]:
from datetime import datetime

sample_submission = pd.read_csv('./submit/sample_submission.csv')
sample_submission['probability'] = test_preds
display(sample_submission)

,ID,probability
0,TEST_00000,0.001617
1,TEST_00001,0.001532
2,TEST_00002,0.156815
3,TEST_00003,0.102414
4,TEST_00004,0.490375
...,...,...
90062,TEST_90062,0.002819
90063,TEST_90063,0.297091
90064,TEST_90064,0.352995
90065,TEST_90065,0.224825


In [163]:
# 제출할 파일명
today = datetime.now().strftime('%m%d_%H%M%S')
file_name = f"./submit/submit_jiho+doeun_{today}.csv"
sample_submission.to_csv(file_name, index=False)